In [ ]:
from pathlib import Path
import datetime as dt
from dateutil.relativedelta import relativedelta
from pytz import timezone
import pandas as pd

from arcgis.gis import GIS

In [ ]:
gis = GIS(profile="your_admin_profile")

### Create users

In [ ]:
data_path = Path(".") / "aux_files/intern_list"

In [ ]:
user_info = data_path / "interns_for_delete.csv"

In [ ]:
user_file = pd.read_csv(user_info)
user_file

In [ ]:
for i in list(user_file[user_file.Role == "Administrator"].index):
    user_file.at[i, "Role"] = "Publisher"

In [ ]:
user_data = user_file.apply(lambda un:un+"@jf62" if un.name == "Username" else un)

In [ ]:
user_data = user_data.apply(lambda pw:pw+"@a7r3" if pw.name == "Password" else pw)

In [ ]:
user_data

In [ ]:
for idx, row in user_data.iterrows():
    try:
        new_user = gis.users.create(username=row["Username"],
                                    password=row["Password"],
                                    firstname=row["First Name"],
                                    lastname=row["Last Name"],
                                    email=row["Email"],
                                    user_type=row["User Type"],
                                    role=row["Role"])
        print(f"....created {new_user.username}")
        dt_created = dt.datetime.fromtimestamp(new_user.created/1000)
    except Exception as e:
        print(str(e))

### Get the newly created users

In [ ]:
# constuct datetime objects for a time range
end = dt_created + relativedelta(minutes=1)
half_hr = relativedelta(minutes=30)
start = end - half_hr # subtract half hour from the time close to now

# format datetime objects to milliseconds for use in query
start4_qy = int(start.timestamp()*1000)
end4_qy = int(end.timestamp()*1000)

In [ ]:
# Make sure we're only working with non Enterprise-logins
builtin_users = [u for u in gis.users.search(f"created: [{start4_qy} TO {end4_qy}]")
                if len(u.groups) < 2]
builtin_users

### Extend the scenario: group query

In [ ]:
for grp in gis.groups.search(query="African Continent"):
    grp_members = grp.get_members()
    for user in grp_members['users']:
        user = gis.users.get(user)
        user.reset(reset_by_email=True)
        print(f"...sent reset password email to {user.username}")

## Reset the password with an email to the users

In [ ]:
for user in builtin_users:
    user.reset(reset_by_email=True)

### Delete the created users

In [ ]:
lic_mgr = gis.admin.license

for user in builtin_users:
    user.disable()
    try:
        user.delete()
        print(f"...deleted {user.username}")
    except Exception as e:
        if user.items() or [i for f in user.folders 
                            for i in user.items(folder=f)] or gis.groups.search(f"owner:{user.username}"):
            user.reassign_to(gis.users.me.username)
            print(f"...reassigning items and/or groups from {user.username}")
        add_on_provisions = user.provisions
        if add_on_provisions:
            for provision in add_on_provisions:
                provision_license = lic_mgr.get(provision.title)
                provision_license.revoke(user.username, entitlements="*")
        user_bundles = user.bundles
        if user_bundles:
            for bundle in user_bundles:
                bundle.revoke(user)
        user.delete()
        print(f"...deleted {user.username}")
        print(f"\n")